In [3]:
import torch
import nltk
import numpy as np
import pandas as pd
from torch import nn
from tqdm import tqdm
from torch.optim import Adam
import matplotlib.pyplot as plt
from transformers import BertTokenizer, BertModel, AutoTokenizer
from sklearn.model_selection import train_test_split, learning_curve
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

plt.style.use('seaborn')
nltk.download('stopwords')
nltk.download('punkt')
BASE_PATH = "D:\\University\\7 Fall 2023\\CMSC516\\Project 1\\Datasets\\"

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\majd2\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\majd2\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
dataset = pd.read_csv(BASE_PATH + "preprocessed_dataset.csv", encoding='utf_8')
dataset.dropna(inplace=True)
dataset.reset_index(drop=True, inplace=True)

In [5]:
max_len = 0
for tweet in dataset['tweet']:
    max_len = max(len(tweet), max_len)
print(max_len)

tokenizer = BertTokenizer.from_pretrained('bert-base-cased', force_download=True)
labels_dict = {'negative':0, 'positive':1}

245


In [8]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):
        self.labels = [labels_dict[label] for label in df['label']]
        self.texts  = [tokenizer(text, padding='max_length', max_length = 250, 
                      truncation=True, return_tensors="pt") for text in df['tweet']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):
        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y